In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import BeamDynamics as bd
import SimulationData as sd
import copy
import json

In [ ]:
from importlib import reload
reload(bd)

In [ ]:
# %matplotlib inline
# %matplotlib notebook
%matplotlib widget
plt.rcParams['figure.figsize'] = [9.6, 6.4]
defaultColorCycle = plt.rcParams["axes.prop_cycle"].by_key()['color']
# plotFont = {
#     'family' : 'sans-serif',
#     'weight' : 'normal',
#     'size'   : 12
# }
# matplotlib.rc('font', **plotFont)
# plt.rc('legend', fontsize=10)

# Distributions from Capture Linac (Approx. 200 MeV)

## Plot Single Distributions

### 200 MeV, Homogeneous Solenoidal Channel 0.5 T, CLIC L-band TW Structures (Yongke, V1)

#### First Positron Buckets (Overall Picture)

In [ ]:
DISTR_REL_PATH = 'SimulationRuns/DistrsFromExternalPartners/PositronsAt200MeV/YongkeDistrsV1/CTSB-N02-F100-E06-S0.5-T5.0_HTSTest_JNov04_SolC_CLICTW-Ztc200-Ri15-Bc0.50.dat'
FILTER_SPECS_SELECTOR = '../../Data/PositronsAt200MeV/YongkeDistrsV1CTSB-N02-F100-E06-S0.5-T5.0_HTSTest_JNov04_SolC_CLICTW-Ztc200-Ri15-Bc0.50_FirstBunches'
# FILTER_SPECS_SELECTOR = None
distrPath = sd.build_data_path(DISTR_REL_PATH)
beamInYonClicZoomOut, _ = bd.convert_rftrack_to_standard_df(
    sourceFilePath=distrPath, sourceFormat='rftrackYongke1', rftrackDfFormat='rftrack_xp_t',
    filterSpecsSelector=FILTER_SPECS_SELECTOR, s=np.nan, pdgId=-11
)
beamInYonClicZoomOut['z'] = beamInYonClicZoomOut.loc[0, 't'] * bd.C / 1e6   # This is very approximative
beamInYonClicZoomOut.describe()

In [ ]:
plotDefs = bd.set_plot_defs_from_distrs([beamInYonClicZoomOut], setName='angles-t')
_ = bd.plot_distr([beamInYonClicZoomOut], plotDefs)

#### Bucket of Interest (1st or 2nd?, Positrons)

In [ ]:
DISTR_REL_PATH = 'SimulationRuns/DistrsFromExternalPartners/PositronsAt200MeV/YongkeDistrsV1/CTSB-N02-F100-E06-S0.5-T5.0_HTSTest_JNov04_SolC_CLICTW-Ztc200-Ri15-Bc0.50.dat'
FILTER_SPECS_SELECTOR = 'CTSB-N02-F100-E06-S0.5-T5.0_HTSTest_JNov04_SolC_CLICTW-Ztc200-Ri15-Bc0.50_MainBunch'
distrPath = sd.build_data_path(DISTR_REL_PATH)
beamInYonClic, _ = bd.convert_rftrack_to_standard_df(
    sourceFilePath=distrPath, sourceFormat='rftrackYongke1', rftrackDfFormat='rftrack_xp_t',
    filterSpecsSelector=FILTER_SPECS_SELECTOR, s=np.nan, pdgId=-11
)
beamInYonClic['z'] = beamInYonClic.loc[0, 't'] * bd.C / 1e6   # This is very approximative
beamInYonClic.describe()

In [ ]:
plotDefs = bd.set_plot_defs_from_distrs([beamInYonClic], setName='angles-t')
_ = bd.plot_distr([beamInYonClic], plotDefs)

### 200 MeV, Homogeneous Solenoidal Channel 1.5 T, PSI S-band SW Structures (Yongke, V1)

#### First Positron Buckets (Overall Picture)

In [ ]:
DISTR_REL_PATH = 'SimulationRuns/DistrsFromExternalPartners/PositronsAt200MeV/YongkeDistrsV1/CTSB-N02-F100-E06-S0.5-T5.0_HTSTest_JNov04_SolC_PSISW-Ztc200-Ri15-Bc1.50.dat'
FILTER_SPECS_SELECTOR = 'CTSB-N02-F100-E06-S0.5-T5.0_HTSTest_JNov04_SolC_PSISW-Ztc200-Ri15-Bc1.50_FirstBunches'
distrPath = sd.build_data_path(DISTR_REL_PATH)
beamInYonPsiZoomOut, _ = bd.convert_rftrack_to_standard_df(
    sourceFilePath=distrPath, sourceFormat='rftrackYongke1', rftrackDfFormat='rftrack_xp_t',
    filterSpecsSelector=FILTER_SPECS_SELECTOR, s=np.nan, pdgId=-11
)
beamInYonPsiZoomOut['z'] = beamInYonPsiZoomOut.loc[0, 't'] * bd.C / 1e6   # This is very approximative
beamInYonPsiZoomOut.describe()

In [ ]:
plotDefs = bd.set_plot_defs_from_distrs([beamInYonPsiZoomOut], setName='angles-t')
_ = bd.plot_distr([beamInYonPsiZoomOut], plotDefs)

#### Bucket of Interest (1st or 2nd?, Positrons)

In [ ]:
DISTR_REL_PATH = 'SimulationRuns/DistrsFromExternalPartners/PositronsAt200MeV/YongkeDistrsV1/CTSB-N02-F100-E06-S0.5-T5.0_HTSTest_JNov04_SolC_PSISW-Ztc200-Ri15-Bc1.50.dat'
FILTER_SPECS_SELECTOR = 'CTSB-N02-F100-E06-S0.5-T5.0_HTSTest_JNov04_SolC_PSISW-Ztc200-Ri15-Bc1.50_MainBunch'
distrPath = sd.build_data_path(DISTR_REL_PATH)
beamInYonPsi, _ = bd.convert_rftrack_to_standard_df(
    sourceFilePath=distrPath, sourceFormat='rftrackYongke1', rftrackDfFormat='rftrack_xp_t',
    filterSpecsSelector=FILTER_SPECS_SELECTOR, s=np.nan, pdgId=-11
)
beamInYonPsi['z'] = beamInYonPsi.loc[0, 't'] * bd.C / 1e6   # This is very approximative
beamInYonPsi.describe()

In [ ]:
plotDefs = bd.set_plot_defs_from_distrs([beamInYonPsi], setName='angles-t')
_ = bd.plot_distr([beamInYonPsi], plotDefs)

### 200 MeV, Realistic Solenoidal Channel 1.5 T, PSI S-band SW Structures, Main bucket is 2nd (Nicolás)

#### First Buckets, Positrons and Electrons (Overall Picture)

In [ ]:
DISTR_REL_PATH = 'SimulationRuns/ASTRA/000012/RUN_1911_164058/RUN_1911_164058.1380.001'
FILTER_SPECS_SELECTOR = 'RUN_1911_164058.1380.001_FirstBunches'
# FILTER_SPECS_SELECTOR = None
zProjection = None
distrPath = sd.build_data_path(DISTR_REL_PATH)
beamInNicoPsi1p5TZoomOut, _ = bd.convert_astra_to_standard_df(
    distrPath, filterSpecsSelector=FILTER_SPECS_SELECTOR,
    zProjection=zProjection, removeNanInf=True
)
beamInNicoPsi1p5TPositronsZoomOut = beamInNicoPsi1p5TZoomOut[beamInNicoPsi1p5TZoomOut['pdgId']==-11]
beamInNicoPsi1p5TPositronsZoomOut.describe()

In [ ]:
beamInNicoPsi1p5TElectronsZoomOut = beamInNicoPsi1p5TZoomOut[beamInNicoPsi1p5TZoomOut['pdgId']==11]
beamInNicoPsi1p5TElectronsZoomOut.describe()

In [ ]:
beamInAllBuckets = [beamInNicoPsi1p5TPositronsZoomOut, beamInNicoPsi1p5TElectronsZoomOut]
plotDefs = bd.set_plot_defs_from_distrs(beamInAllBuckets, setName='angles-z')
_ = bd.plot_distr(beamInAllBuckets, plotDefs, legendLabels=['Positrons', 'Electrons'])

#### Bucket of Interest (2nd, Positrons)

In [ ]:
DISTR_REL_PATH = 'SimulationRuns/ASTRA/000012/RUN_1911_164058/RUN_1911_164058.1380.001'
FILTER_SPECS_SELECTOR = 'RUN_1911_164058.1380.001_MainBunch'
# FILTER_SPECS_SELECTOR = None
zProjection = 13800.
distrPath = sd.build_data_path(DISTR_REL_PATH)
beamInNicoPsi1p5T, _ = bd.convert_astra_to_standard_df(
    distrPath, filterSpecsSelector=FILTER_SPECS_SELECTOR,
    zProjection=zProjection, removeNanInf=True
)
beamInNicoPsi1p5T = beamInNicoPsi1p5T[beamInNicoPsi1p5T['pdgId']==-11]
beamInNicoPsi1p5T.describe()

In [ ]:
plotDefs = bd.set_plot_defs_from_distrs([beamInNicoPsi1p5T], setName='angles-t')
_ = bd.plot_distr([beamInNicoPsi1p5T], plotDefs)

### 200 MeV, Realistic Solenoidal Channel 0.3 T, PSI S-band SW Structures, Main bucket is 2nd (Nicolás)

#### First Buckets, Positrons and Electrons (Overall Picture)

In [ ]:
DISTR_REL_PATH = 'SimulationRuns/ASTRA/000012/RUN_2211_111717/RUN_2211_111717.1380.001'
FILTER_SPECS_SELECTOR = 'RUN_2211_111717.1380.001_FirstBunches'
# FILTER_SPECS_SELECTOR = None
zProjection = None
distrPath = sd.build_data_path(DISTR_REL_PATH)
beamInNicoPsi0p3TZoomOut, _ = bd.convert_astra_to_standard_df(
    distrPath, filterSpecsSelector=FILTER_SPECS_SELECTOR,
    zProjection=zProjection, removeNanInf=True
)
beamInNicoPsi0p3TPositronsZoomOut = beamInNicoPsi0p3TZoomOut[beamInNicoPsi0p3TZoomOut['pdgId']==-11]
beamInNicoPsi0p3TPositronsZoomOut.describe()

In [ ]:
beamInNicoPsi0p3TElectronsZoomOut = beamInNicoPsi0p3TZoomOut[beamInNicoPsi0p3TZoomOut['pdgId']==11]
beamInNicoPsi0p3TElectronsZoomOut.describe()

In [ ]:
beamInAllBuckets = [beamInNicoPsi0p3TPositronsZoomOut, beamInNicoPsi0p3TElectronsZoomOut]
plotDefs = bd.set_plot_defs_from_distrs(beamInAllBuckets, setName='angles-z')
_ = bd.plot_distr(beamInAllBuckets, plotDefs, legendLabels=['Positrons', 'Electrons'])

#### Bucket of Interest (2nd, Positrons)

In [ ]:
DISTR_REL_PATH = 'SimulationRuns/ASTRA/000012/RUN_2211_111717/RUN_2211_111717.1380.001'
FILTER_SPECS_SELECTOR = 'RUN_2211_111717.1380.001_MainBunch'
# FILTER_SPECS_SELECTOR = None
zProjection = 13800.
distrPath = sd.build_data_path(DISTR_REL_PATH)
beamInNicoPsi0p3T, _ = bd.convert_astra_to_standard_df(
    distrPath, filterSpecsSelector=FILTER_SPECS_SELECTOR,
    zProjection=zProjection, removeNanInf=True
)
beamInNicoPsi0p3T = beamInNicoPsi0p3T[beamInNicoPsi0p3T['pdgId']==-11]
beamInNicoPsi0p3T.describe()

In [ ]:
plotDefs = bd.set_plot_defs_from_distrs([beamInNicoPsi0p3T], setName='angles-t')
_ = bd.plot_distr([beamInNicoPsi0p3T], plotDefs)

## Superimpose Distributions

* Main bunch
* Time offset to align them in time
* pz offset to better compare shapes

In [ ]:
distrList = [beamInYonClic, beamInYonPsi, beamInNicoPsiMain1, beamInNicoPsiMain2]
labelList = ['Yongke, CLIC', 'Yongke, PSI', 'Nicolás, PSI, Main 1st', 'Nicolás, PSI, Main 2nd']
tOffsetList = [62.867, 58.731, 46.169, 46.364]
pzOffsetList = [185., 224., 213., 203.]
distrOffsetList = []
selectedDistrs = [1, 2]
for distr, tOffset, pzOffset in [list(zip(distrList, tOffsetList, pzOffsetList))[sel] for sel in selectedDistrs]:
    tmpDistr = distr.copy()
    tmpDistr['t'] = tmpDistr['t'] - tOffset
    tmpDistr['pz'] = tmpDistr['pz'] - pzOffset
    distrOffsetList.append(tmpDistr)

In [ ]:
labelList = [labelList[sel] for sel in selectedDistrs]
plotDefs = bd.set_plot_defs_from_distrs(distrOffsetList, setName='angles-t')
_ = bd.plot_distr(distrOffsetList, plotDefs, legendLabels=labelList)

## Plot Phase Space Ellipses

### Compute Emittance and Corresponding Ellipses

In [ ]:
emitn = {}
emitGeom = {}
emitTraceSpace = {}
alphaTwiss = {}
betaTwiss = {}
gammaTwiss = {}
for planeName in ('x', 'y'):
    emitn[planeName] = bd.compute_emittance(
        beamIn, planeName, norm='normalized', filterSpecs=filterSpecs
    )
    emitGeom[planeName] = bd.compute_emittance(
        beamIn, planeName, norm='geometric', filterSpecs=filterSpecs
    )
    emitTraceSpace[planeName] = bd.compute_emittance(
        beamIn, planeName, norm='tracespace', filterSpecs=filterSpecs
    )
    alphaTwiss[planeName], betaTwiss[planeName], gammaTwiss[planeName] = bd.compute_twiss(
        beamIn, planeName, filterSpecs=filterSpecs
    )
    print(
        'emitn_{0:s} = {1:.1f} pi mm mrad, emitGeom_{0:s} = {2:.1f} pi mm mrad, emitTraceSpace_{0:s} = {3:.1f} pi mm mrad.'.format(
            planeName, emitn[planeName], emitGeom[planeName], emitTraceSpace[planeName]
        )
    )
    print(
        'alphaTwiss_{0:s} = {1:.3f}, betaTwiss_{0:s} = {2:.3f} mm, gammaTwiss_{0:s} = {3:.3f} 1/mm.'.format(
            planeName, alphaTwiss[planeName], betaTwiss[planeName], gammaTwiss[planeName]
        )
    )

In [ ]:
selFa = 4.
gaussianPortions = {1: 0.6827, 2: 0.9545, 3: 0.9973, 4: 0.999937, 6: 0.99999998}
ellipseSpecs = {
    'x': {'alphaTwiss': alphaTwiss['x'], 'betaTwiss': betaTwiss['x']},
    'y': {'alphaTwiss': alphaTwiss['y'], 'betaTwiss': betaTwiss['y']}
}
for planes in (['x'], ['x', 'y']):
    for Fa in (1., 2., 3., 4., 6.):
        selEllipseSpecs = {k: v for k, v in ellipseSpecs.items() if k in planes}
        distrWithinFaSigma, portionWithinFaSigma = bd.distr_within_ellipse(
            beamIn, Fa**2.*emitTraceSpace['x'], selEllipseSpecs
        )
        print(
            'Portion within {:.1f} sigma: {:.3f} (vs. {:.3f} for Gaussian).'.format(
                Fa, portionWithinFaSigma, gaussianPortions[Fa]
            )
        )

### Reference values for FODO design

In [ ]:
emitnRef = 10e3   # [pi mm mrad]
FaRef = 4.
pRef = 200   # [MeV/c]
betaGammaRef = bd.p_to_beta(pRef,-11) * bd.p_to_gamma(pRef,-11)
betaGammaRef

### Select portion theoretically transported
In the assumption of a monochromatic beam

In [ ]:
refEllipseSpecs = {
    'x': {'alphaTwiss': 0, 'betaTwiss': betaTwiss['x']},
    'y': {'alphaTwiss': 0, 'betaTwiss': betaTwiss['y']}
}
FaList = (4., 3., 2., 1.)
distrWithinRef = []
for Fa in FaList:
    distrWithinRef.append(bd.distr_within_ellipse(
        beamIn, Fa**2.*emitnRef/betaGammaRef, refEllipseSpecs
    )[0])

In [ ]:
ax = bd.plot_distr(
    #[beamInYonClic, beamInYonPsi, beamInNicoPsi], plotDefs,
    [beamInNicoPsi], plotDefs,
    # [beamIn, *distrWithinRef], plotDefs,
    # legendLabels=[
    #     'Full distr.',
    #     *['Within {:.1f} sigma'.format(Fa) for Fa in FaList]
    # ]
)
# for ind, Fa in enumerate(FaList):
#     bd.plot_ellipse(
#         ax[1][0,0], Fa**2.*emitnRef/betaGammaRef, semiAxisOrder=2,
#         alphaTwiss=0, betaTwiss=betaTwiss['x'], color=ax[1][0,0].get_children()[ind+1].get_edgecolor()
#     )
#     bd.plot_ellipse(
#         ax[2][0,0], Fa**2.*emitnRef/betaGammaRef, semiAxisOrder=2,
#         alphaTwiss=0, betaTwiss=betaTwiss['y'], color=ax[2][0,0].get_children()[ind+1].get_edgecolor()
#     )

# Positron Distributions after Linac (Approx. 1.5 GeV)

## Load Distributions

### Positrons, 1.5 GeV, Initial Distribution Yongke CLIC (Mattia)

In [ ]:
distrPath = '/home/tia/Repos/GIT_PSIPositronProduction/DistrOut_Linac1_Section1_Simple_1p5GeV.sdf_txt'
beamOutMattia = bd.load_standard_fwf(distrPath)
beamOutMattia.describe()

In [ ]:
plotDefs = bd.set_plot_defs_from_distrs([beamOutMattia], setName='angles-z')
for defInd in [3, 4]:
    plotDefs[defInd]['lims1'] = (102310., 102340.)
    plotDefs[defInd]['lims2'] = (1300., 1550.)
_ = bd.plot_distr([beamOutMattia], plotDefs)

### Positrons, 700 MeV, Full ASTRA Tracking (Nicolás)

In [ ]:
DISTR_REL_PATH = 'SimulationRuns/ASTRA/000014/RUN_2801_132916/RUN_2801_132916.4800.001'
FILTER_SPECS_SELECTOR = 'RUN_2801_132916.4800.001_MainBunch'
# FILTER_SPECS_SELECTOR = None
zProjection = None
distrPath = sd.build_data_path(DISTR_REL_PATH)
beamOutNicoPsi, _ = bd.convert_astra_to_standard_df(
    distrPath, filterSpecsSelector=FILTER_SPECS_SELECTOR,
    zProjection=zProjection
)
beamOutNicoPsi.describe()

In [ ]:
plotDefs = bd.set_plot_defs_from_distrs([beamOutNicoPsi], setName='angles-z')
for defInd in [3, 4]:
    # plotDefs[defInd]['lims1'] = (102310., 102340.)
    plotDefs[defInd]['lims2'] = (500., 750.)
_ = bd.plot_distr([beamOutNicoPsi], plotDefs)

# Distribution at Injector Start (Input Yongke's Tool)

In [ ]:
DISTR_PATH = '/home/tia/Repos/GIT_PSIPositronProduction/RFTrack/YongkeTool/input/HTS_5coils_CLIC_Lband.dat'
SOURCE_FORMAT = 'rftrackOctaveSingleMatrix'
RFTRACK_FORMAT = 'rftrack_xp_t'
beamAfterAmdClicYongke, _ = bd.convert_rftrack_to_standard_df(
    sourceFilePath=DISTR_PATH, sourceFormat=SOURCE_FORMAT, rftrackDfFormat=RFTRACK_FORMAT,
    s=np.nan, pdgId=-11
)
# TODO: Ask exact value to Yongke
beamAfterAmdClicYongke['z'] = beamAfterAmdClicYongke.loc[0, 't'] * bd.C / 1e6   # [mm]

In [ ]:
plotDefs = bd.set_plot_defs_from_distrs([beamAfterAmdClicYongke], setName='angles-t')
# for defInd in [3, 4]:
#     # plotDefs[defInd]['lims1'] = (102310., 102340.)
#     plotDefs[defInd]['lims2'] = (500., 750.)
_ = bd.plot_distr([beamAfterAmdClicYongke], plotDefs)

In [ ]:
DISTR_PATH = '/home/tia/Repos/GIT_PSIPositronProduction/RFTrack/YongkeTool/input/HTS_5coils_PSI_Sband.dat'
SOURCE_FORMAT = 'rftrackOctaveSingleMatrix'
RFTRACK_FORMAT = 'rftrack_xp_t'
beamAfterAmdPsiYongke, _ = bd.convert_rftrack_to_standard_df(
    sourceFilePath=DISTR_PATH, sourceFormat=SOURCE_FORMAT, rftrackDfFormat=RFTRACK_FORMAT,
    s=np.nan, pdgId=-11
)
# TODO: Ask exact value to Yongke
beamAfterAmdPsiYongke['z'] = beamAfterAmdPsiYongke.loc[0, 't'] * bd.C / 1e6   # [mm]

In [ ]:
plotDefs = bd.set_plot_defs_from_distrs([beamAfterAmdPsiYongke], setName='angles-t')
# for defInd in [3, 4]:
#     # plotDefs[defInd]['lims1'] = (102310., 102340.)
#     plotDefs[defInd]['lims2'] = (500., 750.)
_ = bd.plot_distr([beamAfterAmdPsiYongke], plotDefs)

In [ ]:
distrList = [beamAfterAmdClicYongke, beamAfterAmdPsiYongke]
labelList = ['Yongke, CLIC', 'Yongke, PSI']
# tOffsetList = [62.867, 58.731, 46.169, 46.364]
# pzOffsetList = [185., 224., 213., 203.]
# distrOffsetList = []
# selectedDistrs = [0, 1]
# for distr, tOffset, pzOffset in [list(zip(distrList, tOffsetList, pzOffsetList))[sel] for sel in selectedDistrs]:
#     tmpDistr = distr.copy()
#     tmpDistr['t'] = tmpDistr['t'] - tOffset
#     tmpDistr['pz'] = tmpDistr['pz'] - pzOffset
#     distrOffsetList.append(tmpDistr)

In [ ]:
# labelList = [labelList[sel] for sel in selectedDistrs]
plotDefs = bd.set_plot_defs_from_distrs(distrList, setName='angles-t')
_ = bd.plot_distr(distrList, plotDefs, legendLabels=labelList)

<div class="alert alert-block alert-success">
Some good news.
</div>

<div class="alert alert-block alert-warning">
Some warning.
</div>

<div class="alert alert-block alert-danger">
Some danger.
</div>